# Iris 분류

### Logistic Regression

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()

In [3]:
# data, target, target_names, feature_names, DESCR
data = iris.data
label = iris.target
columns = iris.feature_names
df = pd.DataFrame(data, columns=columns)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = \
    train_test_split(data, label, test_size=0.2, 
                     shuffle=True, stratify=label, random_state=2019)

* 모델 객체 생성

In [5]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

* 모델 학습

In [6]:
lr.fit(x_train, y_train)

LogisticRegression()

* 학습한 결과를 저장

In [7]:
import joblib
joblib.dump(lr, 'model/iris_lr.pkl')

['model/iris_lr.pkl']

### Support Vector Machine

In [13]:
from sklearn.svm import SVC

In [14]:
# 모델 객체 생성
svc = SVC()

In [15]:
# 학습
svc.fit(x_train, y_train)

SVC()

In [21]:
import joblib
joblib.dump(lr, 'model/iris_svm.pkl')

['model/iris_svm.pkl']

### Decision Tree

In [17]:
from sklearn.tree import DecisionTreeClassifier

In [18]:
dtc = DecisionTreeClassifier()

In [19]:
dtc.fit(x_train, y_train)

DecisionTreeClassifier()

In [22]:
import joblib
joblib.dump(lr, 'model/iris_dt.pkl')

['model/iris_dt.pkl']

### 저장된 모델로 정확도 산출

In [25]:
model_lr = joblib.load('model/iris_lr.pkl')
model_svm = joblib.load('model/iris_svm.pkl')
model_dt = joblib.load('model/iris_dt.pkl')

In [27]:
y_pred_lr = model_lr.predict(x_test)
y_pred_svm = model_svm.predict(x_test)
y_pred_dt = model_dt.predict(x_test)

In [29]:
from sklearn.metrics import accuracy_score
acc_lr = accuracy_score(y_test, y_pred_lr)
acc_svm = accuracy_score(y_test, y_pred_svm)
acc_dt = accuracy_score(y_test, y_pred_dt)
print(acc_lr, acc_svm, acc_dt)

0.9666666666666667 0.9666666666666667 0.9666666666666667


### 한개의 테스트 값 예측

In [33]:
df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3
149,5.9,3.0,5.1,1.8


In [35]:
test_data = np.array([6.7, 3.0, 5.2, 2.3]).reshape(1, 4)

In [37]:
index_lr = model_lr.predict(test_data)[0]
index_svm = model_svm.predict(test_data)[0]
index_dt = model_dt.predict(test_data)[0]

In [39]:
sp_names = ['Setosa', 'Versicolor', 'Virginica']
print(sp_names[index_lr], sp_names[index_svm], sp_names[index_dt])

Virginica Virginica Virginica


# Deep Learning Model

In [63]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
    Dense(12, input_shape=(4,), activation='relu'),
    Dense(8, input_shape=(4,), activation='relu'),
    Dense(3, activation='softmax')
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 12)                60        
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 27        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


In [64]:
# 모델 컴파일
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [65]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath="model/iris_deep.hdf5", monitor='val_loss', 
                               verbose=1, save_best_only=True)

In [66]:
# One hot encoding
Y_encoded = keras.utils.to_categorical(y_train)
Y_encoded[0], Y_encoded[50], Y_encoded[100],

(array([1., 0., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32),
 array([0., 0., 1.], dtype=float32))

In [67]:
model.fit(x_train, Y_encoded, epochs=100, batch_size=30, verbose=2,
          validation_split=0.2, callbacks=[checkpointer])

Train on 96 samples, validate on 24 samples
Epoch 1/100
 - 0s - loss: 1.2302 - accuracy: 0.3438 - val_loss: 1.2786 - val_accuracy: 0.2917

Epoch 00001: val_loss improved from inf to 1.27863, saving model to model/iris_deep.hdf5
Epoch 2/100
 - 0s - loss: 1.1908 - accuracy: 0.3438 - val_loss: 1.2328 - val_accuracy: 0.2917

Epoch 00002: val_loss improved from 1.27863 to 1.23284, saving model to model/iris_deep.hdf5
Epoch 3/100
 - 0s - loss: 1.1581 - accuracy: 0.3438 - val_loss: 1.1908 - val_accuracy: 0.2917

Epoch 00003: val_loss improved from 1.23284 to 1.19082, saving model to model/iris_deep.hdf5
Epoch 4/100
 - 0s - loss: 1.1247 - accuracy: 0.3438 - val_loss: 1.1533 - val_accuracy: 0.2917

Epoch 00004: val_loss improved from 1.19082 to 1.15327, saving model to model/iris_deep.hdf5
Epoch 5/100
 - 0s - loss: 1.0997 - accuracy: 0.3438 - val_loss: 1.1182 - val_accuracy: 0.3333

Epoch 00005: val_loss improved from 1.15327 to 1.11819, saving model to model/iris_deep.hdf5
Epoch 6/100
 - 0s - 


Epoch 00044: val_loss improved from 0.83190 to 0.82415, saving model to model/iris_deep.hdf5
Epoch 45/100
 - 0s - loss: 0.7825 - accuracy: 0.8542 - val_loss: 0.8180 - val_accuracy: 0.7917

Epoch 00045: val_loss improved from 0.82415 to 0.81798, saving model to model/iris_deep.hdf5
Epoch 46/100
 - 0s - loss: 0.7755 - accuracy: 0.8854 - val_loss: 0.8122 - val_accuracy: 0.7917

Epoch 00046: val_loss improved from 0.81798 to 0.81220, saving model to model/iris_deep.hdf5
Epoch 47/100
 - 0s - loss: 0.7689 - accuracy: 0.8750 - val_loss: 0.8071 - val_accuracy: 0.7917

Epoch 00047: val_loss improved from 0.81220 to 0.80715, saving model to model/iris_deep.hdf5
Epoch 48/100
 - 0s - loss: 0.7622 - accuracy: 0.8646 - val_loss: 0.8024 - val_accuracy: 0.7917

Epoch 00048: val_loss improved from 0.80715 to 0.80240, saving model to model/iris_deep.hdf5
Epoch 49/100
 - 0s - loss: 0.7558 - accuracy: 0.8646 - val_loss: 0.7973 - val_accuracy: 0.7917

Epoch 00049: val_loss improved from 0.80240 to 0.79726


Epoch 00088: val_loss improved from 0.66529 to 0.66063, saving model to model/iris_deep.hdf5
Epoch 89/100
 - 0s - loss: 0.6027 - accuracy: 0.9167 - val_loss: 0.6580 - val_accuracy: 0.9167

Epoch 00089: val_loss improved from 0.66063 to 0.65796, saving model to model/iris_deep.hdf5
Epoch 90/100
 - 0s - loss: 0.5998 - accuracy: 0.9271 - val_loss: 0.6567 - val_accuracy: 0.9167

Epoch 00090: val_loss improved from 0.65796 to 0.65671, saving model to model/iris_deep.hdf5
Epoch 91/100
 - 0s - loss: 0.5969 - accuracy: 0.9271 - val_loss: 0.6550 - val_accuracy: 0.9167

Epoch 00091: val_loss improved from 0.65671 to 0.65500, saving model to model/iris_deep.hdf5
Epoch 92/100
 - 0s - loss: 0.5945 - accuracy: 0.9062 - val_loss: 0.6514 - val_accuracy: 0.9167

Epoch 00092: val_loss improved from 0.65500 to 0.65145, saving model to model/iris_deep.hdf5
Epoch 93/100
 - 0s - loss: 0.5922 - accuracy: 0.9271 - val_loss: 0.6471 - val_accuracy: 0.9167

Epoch 00093: val_loss improved from 0.65145 to 0.64709

In [71]:
from keras.models import load_model
model_deep = load_model("model/iris_deep.hdf5")

In [73]:
# 결과 출력 
model_deep.evaluate(x_test, keras.utils.to_categorical(y_test))[1]

30/30 [==============================] - 0s 465us/step


0.9666666388511658

In [76]:
model_deep.predict_classes(test_data)[0]

2

In [77]:
model_deep.predict(test_data)

array([[0.23185335, 0.35101488, 0.41713175]], dtype=float32)

In [75]:
np.argmax(model_deep.predict(test_data)[0])

2